In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## View Information on Database

In [13]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Analyze Genre/Album/Artist Popularity

We see that the most popular music genre is Rock in the US, and it is by a lot. More than 50% of tracks sold in the US are from this genre. Other albums on the top 3 list are alternative & punk, and metal

Recommendations based on this information include to stock the store with mostly Rock music albums

In [89]:
%%sql
SELECT
    name,
    quantity,
    CAST(quantity AS FLOAT) / total AS pct_quantity
FROM (
    WITH nq AS (
        SELECT
            genre.name,
            SUM(invoice_line.quantity) AS quantity
        FROM genre
        LEFT JOIN track ON track.genre_id = genre.genre_id
        LEFT JOIN invoice_line ON invoice_line.track_id = track.track_id
        LEFT JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
        LEFT JOIN customer ON customer.customer_id = invoice.customer_id
        WHERE LOWER(customer.country) = "usa"
        GROUP BY genre.genre_id
    )
    SELECT
        nq.name,
        nq.quantity,
        CASE WHEN nq.quantity NOT NULL THEN (
            SELECT SUM(nq.quantity)
            FROM nq
        )
        END  AS total
    FROM nq AS nq
)
ORDER BY pct_quantity DESC

Done.


name,quantity,pct_quantity
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Let's see the most popular artists in the top 3 most popular genres in the US

In [109]:
%%sql
WITH nq AS (
        SELECT
            album.title AS album_title,
            artist.name AS artist_name,
            genre.name AS genre,
            track.name AS track_name,
            customer.country,
            SUM(invoice_line.quantity) AS quantity
        FROM genre
        LEFT JOIN track ON track.genre_id = genre.genre_id
        LEFT JOIN album ON album.album_id = track.album_id
        LEFT JOIN artist ON artist.artist_id = album.artist_id
        LEFT JOIN invoice_line ON invoice_line.track_id = track.track_id
        LEFT JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
        LEFT JOIN customer ON customer.customer_id = invoice.customer_id
        WHERE
            LOWER(customer.country) = "usa"
            AND genre.name IN ("Rock", "Alternative & Punk", "Metal")
        GROUP BY album.album_id
    )
SELECT
    *
FROM nq
GROUP BY album_title
ORDER BY quantity DESC

Done.


album_title,artist_name,genre,track_name,country,quantity
Are You Experienced?,Jimi Hendrix,Rock,Highway Chile,USA,28
From The Muddy Banks Of The Wishkah [live],Nirvana,Rock,Blew,USA,28
The Doors,The Doors,Rock,The End,USA,27
Green,R.E.M.,Alternative & Punk,Untitled,USA,25
"Hot Rocks, 1964-1971 (Disc 1)",The Rolling Stones,Rock,Let's Spend The Night Together,USA,25
Use Your Illusion I,Guns N' Roses,Rock,Coma,USA,23
Mezmerize,System Of A Down,Metal,Lost in Hollywood,USA,22
20th Century Masters - The Millennium Collection: The Best of Scorpions,Scorpions,Rock,Send Me an Angel,USA,21
Facelift,Alice In Chains,Rock,Real Thing,USA,21
"The Best Of Van Halen, Vol. I",Van Halen,Rock,Me Wise Magic,USA,21


From the table above, we see that the most popular albums are Jimi Hendrix's "Are You Experienced?" - a rock album, followed by "From The Muddy Baks Of the Wishkah. We can use the results from above to get the top three albums as shown below.

The three artists and their albums for which we should purchase are;
- Jimi Hendrix - Are You Experienced?
- Nirvana - From The Muddy Banks Of The Wishkah [live]
- The Doors - The Doors

In [112]:
%%sql
WITH nq AS (
        SELECT
            album.title AS album_title,
            artist.name AS artist_name,
            genre.name AS genre,
            track.name AS track_name,
            customer.country,
            SUM(invoice_line.quantity) AS quantity
        FROM genre
        LEFT JOIN track ON track.genre_id = genre.genre_id
        LEFT JOIN album ON album.album_id = track.album_id
        LEFT JOIN artist ON artist.artist_id = album.artist_id
        LEFT JOIN invoice_line ON invoice_line.track_id = track.track_id
        LEFT JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
        LEFT JOIN customer ON customer.customer_id = invoice.customer_id
        WHERE
            LOWER(customer.country) = "usa"
            AND genre.name IN ("Rock", "Alternative & Punk", "Metal")
        GROUP BY album.album_id
    )
SELECT
    *
FROM nq
GROUP BY album_title
ORDER BY quantity DESC
LIMIT 3;

Done.


album_title,artist_name,genre,track_name,country,quantity
Are You Experienced?,Jimi Hendrix,Rock,Highway Chile,USA,28
From The Muddy Banks Of The Wishkah [live],Nirvana,Rock,Blew,USA,28
The Doors,The Doors,Rock,The End,USA,27


## Analyze Sales Agent Performance

Here, we see that Jane Peacock is the top performing agent as she has made sales fo mroe than 1700. Steve, on the other hand made sales of slightly less than 1400. It is worth noting that a lot of other variables could be in play affecting these figures, therefore I will be analyzing them shortly

In [137]:
%%sql
SELECT
    e.first_name || " " || e.last_name agent_name,
    SUM(i.total) AS total_sales
FROM employee AS e
LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
WHERE e.title = "Sales Support Agent"
GROUP BY employee_id
ORDER BY total_sales DESC

Done.


agent_name,total_sales
Jane Peacock,1731.510000000004
Margaret Park,1584.0000000000032
Steve Johnson,1393.9200000000028


Here we see that the number of customers forwarded to the agents wasn't actually that different for the "top 2" best performing. For Steve, however, the difference is relatively big, therefore we can't outrightly say he was the least performing in terms of sales made. Next up we will see the countries where Margaret and Jane's customers were from

In [148]:
%%sql
SELECT
    e.first_name || " " || e.last_name agent_name,
    COUNT(c.customer_id) AS customer_count
FROM employee AS e
LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
WHERE e.title = "Sales Support Agent"
GROUP BY e.employee_id
ORDER BY customer_count DESC

Done.


agent_name,customer_count
Margaret Park,214
Jane Peacock,212
Steve Johnson,188


Most of Margaret's customers came from the US. This is not really an advantage. I will show later on that even though most customers come from the US, the average spending per customer from teh US is not actually the best. So it means this doesn't really give Margaret an edge over Jane in terms of the likelihood of customers paying. We could argue that Margaret is just good compared to Jane at her craft

In [166]:
%%sql
SELECT *
FROM (
    SELECT
        e.first_name || " " || e.last_name agent_name,
        c.country,
        COUNT(e.employee_id) AS country_count
    FROM employee AS e
    LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
    LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
    WHERE e.title = "Sales Support Agent"
    GROUP BY c.country, e.employee_id
    ORDER BY country_count DESC
)
WHERE agent_name = "Margaret Park"

Done.


agent_name,country,country_count
Margaret Park,USA,63
Margaret Park,Portugal,29
Margaret Park,Brazil,27
Margaret Park,Czech Republic,18
Margaret Park,France,18
Margaret Park,Australia,10
Margaret Park,Denmark,10
Margaret Park,Poland,10
Margaret Park,Norway,9
Margaret Park,Canada,8


In terms of total spending, the US recorded the highest. The figure that follows this one though that the average spending per customer doesn't paint the same picture'

In [167]:
%%sql
SELECT *
FROM (
    SELECT
        c.country,
        SUM(i.total) total_spending
    FROM customer as c
    LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
    GROUP BY c.country
)
ORDER BY total_spending DESC

Done.


country,total_spending
USA,1040.4900000000005
Canada,535.59
Brazil,427.6799999999999
France,389.07000000000005
Germany,334.61999999999995
Czech Republic,273.23999999999995
United Kingdom,245.52
Portugal,185.13
India,183.14999999999995
Ireland,114.83999999999997


Here we see the countries with highest average spending per customer

In [171]:
%%sql
SELECT *
FROM (
    SELECT
        c.country,
        SUM(i.total) / COUNT(DISTINCT(c.customer_id)) AS average_spending
    FROM customer as c
    LEFT JOIN invoice AS i ON c.customer_id = i.customer_id
    GROUP BY c.country
    HAVING COUNT(DISTINCT(c.customer_id)) > 3
)
ORDER BY average_spending DESC

Done.


country,average_spending
Brazil,85.53599999999997
Germany,83.65499999999999
USA,80.03769230769234
France,77.81400000000001
Canada,66.94875


In [172]:
%%sql
SELECT *
FROM (
    SELECT
        e.first_name || " " || e.last_name agent_name,
        c.country,
        COUNT(e.employee_id) AS country_count
    FROM employee AS e
    LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
    LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
    WHERE e.title = "Sales Support Agent"
    GROUP BY c.country, e.employee_id
    ORDER BY country_count DESC
)
WHERE agent_name = "Jane Peacock"

Done.


agent_name,country,country_count
Jane Peacock,Canada,47
Jane Peacock,USA,25
Jane Peacock,Brazil,24
Jane Peacock,France,23
Jane Peacock,India,21
Jane Peacock,Germany,19
Jane Peacock,United Kingdom,19
Jane Peacock,Ireland,13
Jane Peacock,Finland,11
Jane Peacock,Hungary,10


## Analyzing Sales By Country

In [316]:
%%sql
        SELECT
            c.country,
            SUM(i.total) AS order_value
        FROM customer AS c
        LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
        GROUP BY c.country

Done.


country,order_value
Argentina,39.6
Australia,81.18
Austria,69.3
Belgium,60.38999999999999
Brazil,427.6799999999999
Canada,535.59
Chile,97.02000000000001
Czech Republic,273.23999999999995
Denmark,37.61999999999999
Finland,79.2


In [322]:
%%sql
WITH country_sales AS
    (
        SELECT
            c.country,
            SUM(i.total) AS sales
        FROM customer AS c
        LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
        GROUP BY c.country
    ),
    country_count AS
    (
        SELECT
            c.country,
            CASE WHEN COUNT(c.country) > 1 THEN c.country
                ELSE "Other"
                END AS country_val,
            CASE WHEN COUNT(c.country) > 1 THEN 0
                ELSE 1
                END AS position,
            COUNT(c.country) AS count
        FROM customer AS c
        GROUP BY c.country
    ),
    country_order AS
    (
        SELECT
            c.country,
            SUM(i.total) AS order_value
        FROM customer AS c
        LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
        GROUP BY i.billing_country
    )
SELECT
    count.country_val AS country,
    SUM(count.count) AS customer_count,
    ROUND(SUM(sales.sales), 2) AS total_sales,
    ROUND(SUM(sales.sales) / SUM(count.count), 2) AS sales_per_customer,
    ROUND(orders.order_value, 2) AS order_value
FROM country_count AS count
LEFT JOIN country_sales AS sales ON sales.country = count.country
LEFT JOIN country_order AS orders ON sales.country = orders.country
GROUP BY country_val
ORDER BY position, sales DESC 


Done.


country,customer_count,total_sales,sales_per_customer,order_value
USA,13,1040.49,80.04,1040.49
Canada,8,535.59,66.95,535.59
Brazil,5,427.68,85.54,427.68
France,5,389.07,77.81,389.07
Germany,4,334.62,83.65,334.62
Czech Republic,2,273.24,136.62,273.24
United Kingdom,3,245.52,81.84,245.52
Portugal,2,185.13,92.56,185.13
India,2,183.15,91.57,183.15
Other,15,1094.94,73.0,75.24


## Albums Vs. Individual Tracks

Most people bought a number of tracks and not full albums. Here, we see the difference is overwhelming. over 80% of people bought tracks individually, while less than 20% bought full albums

Since most people buy individual tracks, it would be good if it is made simple for users to utilize this approach

In [396]:
%%sql
WITH track_info AS
    (
        SELECT
            i.invoice_id,
            track.track_id,
            album.album_id
        FROM invoice AS i
        LEFT JOIN invoice_line AS il ON il.invoice_id = i.invoice_id
        LEFT JOIN track ON track.track_id = il.track_id
        LEFT JOIN album ON track.album_id = album.album_id
        ORDER BY 1
    ),
    invoice_info AS
    (
        SELECT
            invoice_id,
            album_id,
            COUNT(DISTINCT(track_id)) AS invoice_distinct_tracks,
            COUNT(DISTINCT(album_id)) AS invoice_distinct_albums
        FROM track_info
        GROUP BY invoice_id   
    ),
    album_info AS
    (
        SELECT
            album.album_id,
            COUNT(DISTINCT(track.track_id)) AS album_distinct_tracks
        FROM album
        LEFT JOIN track ON album.album_id = track.album_id
        GROUP BY album.album_id
    ),
    album_purchase AS
    (
         SELECT
            i.invoice_id,
            CASE WHEN i.invoice_distinct_tracks = a.album_distinct_tracks
            AND  i.invoice_distinct_albums = 1
            THEN 1 ELSE 0 
            END AS album_purchase
        FROM invoice_info AS i
        LEFT JOIN album_info AS a ON i.album_id = a.album_id   
    )
SELECT
    album_purchase,
    COUNT(invoice_id) AS count,
    ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(invoice_id) FROM album_purchase), 2) AS pct_purchase
FROM album_purchase
GROUP BY album_purchase

Done.


album_purchase,count,pct_purchase
0,500,0.81
1,114,0.19
